Copyright(c) 2021 UNIADEX, Ltd. All Rights Reserved.  
CONFIDENTIAL

# ショット切り出し  

以下の処理を行います。  

* ショット切り出し
* 中断区間のデータ除外
* 物理変換 + 校正
* SPM計算
* 事象記録のタグ付け
* SPMから算出される、ショット当たりの最大サンプル数を超えたショット除外
* Elasticsearchインデックスへの保存
  * shots-yyyyMMddHHMMSS-data：切り出されたショットデータ
  * shots-yyyyMMddHHMMSS-meta：ショットのメタデータ
  

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from cut_out_shot.cut_out_shot import CutOutShot
from utils import common
import logging

In [2]:
# ログ設定
LOG_FILE = os.path.join(
    common.get_config_value(common.APP_CONFIG_PATH, "log_dir"), "cut_out_shot/cut_out_shot.log"
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.handlers.RotatingFileHandler(
            LOG_FILE, maxBytes=common.MAX_LOG_SIZE, backupCount=common.BACKUP_COUNT
        ),
        logging.StreamHandler(),
    ],
)

### CutOutShot クラスパラメーター一覧  

**【必須】**  
***displacement_func***  
変位センサーの物理変換式  

***load01_func***  
荷重センサー01の物理変換式  

***load02_func***  
荷重センサー02の物理変換式  

***load03_func***  
荷重センサー03の物理変換式  

***load04_func***  
荷重センサー04の物理変換式  


**【任意】**  
***min_spm***  
最小SPM。この値をもとに、1ショットに含まれる最大サンプル数を計算し、最大サンプル数を超えたショットは切り出し対象外とする。  
指定しない場合、既定値の15となる。  

***back_seconds_for_tagging***  
データ収集マネージャーで記録した事象について、何秒遡ってタグ付けするかを指定。指定しない場合、既定値の120となる。

***previous_size***  
ショットを検知したサンプルから何サンプル遡ってショットに含めるかを指定。これは切り出しショットに荷重開始点が確実に含まれるようにするため。指定しない場合、既定値の1000となる。


### cut_out_shot 関数パラメーター一覧
**【必須】**  
***rawdata_dir_name***  
ショット切り出しの対象とする生データファイルのディレクトリ（.pklファイルが含まれるディレクトリ）。  
ディレクトリ名はデータ収集開始日時で、yyyyMMddHHMMSSの形式となっている。

***start_displacement***  
ショット切り出しの開始を検知するための変位値。  

***end_displacement***  
ショット切り出しの終了を検知するための変位値。  

**【任意】**  
***start_sequential_number***  
ショット切り出しの開始連番。指定しない場合、データの先頭から切り出し。  

***end_sequential_number***  
ショット切り出しの終了連番。指定しない場合、データの末尾まで切り出し。  

In [3]:
# データディレクトリの一覧
!ls /datadrive/data

ls: cannot access '/datadrive/data': No such file or directory


In [3]:
# 物理変換式設定
#displacement_func = lambda v: 70.0 - (v - 2.0) * 70.0 / 8.0
displacement_func = lambda v: v
Vr = 2.5
load01_func = lambda v: 2.5 / Vr * v
load02_func = lambda v: 2.5 / Vr * v
load03_func = lambda v: 2.5 / Vr * v
load04_func = lambda v: 2.5 / Vr * v

In [8]:
# 適宜変更してください
rawdata_dir_name="20210101000000"
#start_displacement=47.0
#end_displacement=-34.0
start_displacement=158.0
end_displacement=-5.0
# start_sequential_number=20000
# end_sequential_number=140000

cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func,
    min_spm=1,
#     margin=0.3,
    previous_size=0
)

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement,
#     start_sequential_number=start_sequential_number,
#     end_sequential_number=end_sequential_number
)

2021-04-06 15:52:17,233 [INFO] Cut out shot start.
2021-04-06 15:52:17,294 [INFO] delete index 'shots-20210101000000-data' finished. result: {'acknowledged': True}
2021-04-06 15:52:17,487 [INFO] create index 'shots-20210101000000-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20210101000000-data'}
2021-04-06 15:52:17,546 [INFO] delete index 'shots-20210101000000-meta' finished. result: {'acknowledged': True}
2021-04-06 15:52:17,742 [INFO] create index 'shots-20210101000000-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20210101000000-meta'}
2021-04-06 15:52:17,757 [ERROR] Invalid spm. shot_number: 1, spm: 6000.0, rawdata_sequential_number: 1194
2021-04-06 15:52:17,763 [ERROR] Invalid spm. shot_number: 2, spm: 6000.0, rawdata_sequential_number: 2386
2021-04-06 15:52:17,769 [ERROR] Invalid spm. shot_number: 3, spm: 6000.0, rawdata_sequential_number: 3578
2021-04-06 15:52:17,775 [ERROR] Invalid spm. 

<br>
<br>
<br>
<br>
<br>
<br>
<hr>

## 参考）テストデータでのショット切り出し

In [9]:
# 適宜変更してください
rawdata_dir_name="20201201010000"
start_displacement=47.0
end_displacement=34.0
start_sequential_number=None
end_sequential_number=None

displacement_func = lambda v: v
load01_func = lambda v: v
load02_func = lambda v: v
load03_func = lambda v: v
load04_func = lambda v: v

cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func
)

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement,
    start_sequential_number=start_sequential_number,
    end_sequential_number=end_sequential_number
)

2021-02-26 15:36:40,338 [INFO] Cut out shot start.
2021-02-26 15:36:40,570 [INFO] delete index 'shots-20201201010000-data' finished. result: {'acknowledged': True}
2021-02-26 15:36:40,770 [INFO] create index 'shots-20201201010000-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-data'}
2021-02-26 15:36:40,830 [INFO] delete index 'shots-20201201010000-meta' finished. result: {'acknowledged': True}
2021-02-26 15:36:41,035 [INFO] create index 'shots-20201201010000-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-meta'}
2021-02-26 15:37:08,879 [INFO] 436180 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp000.pkl.
2021-02-26 15:37:31,596 [INFO] Thoughput: 197795 doc/sec, processed_count: 10000000
2021-02-26 15:37:37,612 [INFO] 436180 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp001.pkl.
202